In [10]:
from utils import Json_parse
from utils import Pre_processing
from utils import run_portfolio_optimization_suite
import json

In [11]:
with open("./small_dataset.json") as f:
  input_data = json.load(f)
data=input_data['data']

price_df, esg_s, asset_arr = Json_parse.json_parse(data)
    # price_df, esg_s, asset_arr = Json_parse.json_parse(get_subdic(dic, size=size))

    # Get mu & sigma
mu, Sigma = Pre_processing.compute_mu_sigma(price_df)

config_example = {
    'compute_inputs': {'shrinkage': True},
    'scalarized_qp': {
        'lamb_grid': [0.5, 1.0, 2.0],
        'eta_grid': [0.0, 0.2, 0.5],
        'lb': 0.0, 'ub': 0.2, 'allow_short': False # Example: Max 20% per asset
    },
    'nsga2': {
        'pop_size': 150, 'generations': 80, 'plot': True
    },
    'cardinality': {
        'k': 15, 'lamb': 1.0, 'eta': 0.2
    },
    'cvar': {
        'alpha': 0.90, 'lamb': 0.5, 'eta': 0.3, 'use_mu_in_obj': True
    }
}
# Assume price_df and esg_df are loaded pandas DataFrames
results_dict = run_portfolio_optimization_suite(price_df, mu, Sigma, asset_arr, esg_s, config_example)
print("\n--- Results Summary ---")
print(results_dict.keys())


Inputs computed for 13 asset_arr.

Running Scalarized QP grid search...
  lambda=0.50, eta=0.00 -> ret=0.3695, vol=0.1291, esg=6.5180
  lambda=0.50, eta=0.20 -> ret=0.1788, vol=0.1084, esg=9.4253
  lambda=0.50, eta=0.50 -> ret=0.0680, vol=0.1179, esg=9.6040
  lambda=1.00, eta=0.00 -> ret=0.3695, vol=0.1291, esg=6.5180
  lambda=1.00, eta=0.20 -> ret=0.1788, vol=0.1084, esg=9.4254
  lambda=1.00, eta=0.50 -> ret=0.1788, vol=0.1084, esg=9.4253
  lambda=2.00, eta=0.00 -> ret=0.3695, vol=0.1291, esg=6.5180
  lambda=2.00, eta=0.20 -> ret=0.2725, vol=0.1159, esg=8.8167
  lambda=2.00, eta=0.50 -> ret=0.1788, vol=0.1084, esg=9.4253

Running CVaR Optimization...
  CVaR alpha=0.9, return lambda=0.50, ESG eta=0.30
  Non-zero weights: 5
  CVaR Metrics: ret=0.1882, vol=0.2036, esg=11.0000

Optimization suite finished.

--- Results Summary ---
dict_keys(['inputs', 'scalarized_qp_results', 'cvar_result'])


/home/jovyan/work/git/singapore25_challenge3_team9/utils/Pre_processing.py:28: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  rets = prices_df.pct_change().dropna() # percentage change between the current element and prior element


In [13]:
results_dict["scalarized_qp_results"]

[{'lambda': 0.5,
  'eta': 0.0,
  'weights': array([0.2, 0. , 0.2, 0. , 0.2, 0. , 0.2, 0. , 0. , 0.2, 0. , 0. , 0. ]),
  'expected_return': 0.36953229914663444,
  'variance': 0.01666537358344413,
  'volatility': np.float64(0.1290944366866525),
  'esg_score': 6.518000000000001},
 {'lambda': 0.5,
  'eta': 0.2,
  'weights': array([-1.38577495e-07,  1.99999720e-01,  1.99999834e-01,  2.93335294e-08,
          1.99999912e-01, -2.25177658e-08,  2.67232921e-08, -2.48563206e-07,
         -1.00294374e-07, -9.33478876e-08,  1.99999950e-01,  0.00000000e+00,
          2.00001132e-01]),
  'expected_return': 0.17882465262635397,
  'variance': 0.011747984939201588,
  'volatility': np.float64(0.10838812176249567),
  'esg_score': 9.425334660739358},
 {'lambda': 0.5,
  'eta': 0.5,
  'weights': array([ 1.69812625e-06,  2.00006231e-01,  1.99998873e-01,  2.00001511e-01,
         -9.60916522e-06, -6.39502757e-07,  2.19285140e-06,  1.63042236e-06,
         -1.41666311e-06, -1.19423147e-06,  2.00006275e-01,  5.